In [2]:
from denovo_utils.parsers import DenovoEngineConverter
from denovo_utils.utils.pandas import collapse_casanovo_score
from denovo_utils.parsers.constants import EXTENSIONS
from denovo_utils.analysis.feature_generation import calculate_hyperscore
from denovo_utils.analysis.missing_fragmentations import MissingFragmentationSiteFGen
from spectrum_utils.utils import mass_diff
from pyteomics import mgf
import pandas as pd
import os
from glob import glob
from tqdm import tqdm
import numpy as np

Modification already exists in ModificationsDB. Skipping.


2024-09-24 10:56:28.989929: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-24 10:56:28.989965: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-24 10:56:28.989984: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from denovo_utils.analysis.missing_fragmentations import get_annotated_spectrum
from denovo_utils.utils.annotation import AnnotatedSpectrum

def psm_to_annotated_spectrum(psm, mgf_file):
    annotated_spec = get_annotated_spectrum(
        mgf_file,
        psm,
        ions="byp",
        neutral_losses=True
    )
    return AnnotatedSpectrum(
        annotated_spec,
        peplen=len(psm["peptidoform"]),
        process=True,
        add_neutral_losses=["-H2O", "-NH3"]
    )

def psmlist_to_annotated_spectra(psmlist, mgf_file, out_path, save=True):
    spec_dict = {}
    for i, psm in enumerate(psmlist):
        spec_dict[i] = {
            "spectrum_id": psm["spectrum_id"],
            "annotated_spectrum": psm_to_annotated_spectrum(
                psm=psm,
                mgf_file=mgf_file
            ),
            "run": psm["run"],
            "source": psm["source"]
        }
    df = pd.DataFrame(spec_dict)

    if save:
        df.to_feather(out_path)
    return df

In [4]:
def calculate_ppms(ion_mz_delta):
    y_ppm = []
    for y in ion_mz_delta["y1"]:
        if y != 0:
            y_ppm.append(y)
    for y in ion_mz_delta["y2"]:
        if y != 0:
            y_ppm.append(y)

    b_ppm = []
    for b in ion_mz_delta["b1"]:
        if b != 0:
            b_ppm.append(b)
    for b in ion_mz_delta["b2"]:
        if b != 0:
            b_ppm.append(b)

    if y_ppm:
        y_ppm_mean = np.mean(y_ppm)
    else:
        y_ppm_mean = 0.
    
    if b_ppm:
        b_ppm_mean = np.mean(b_ppm)
    else:
        b_ppm_mean = 0.
    return b_ppm_mean, y_ppm_mean

def calculate_pct_intensities(tic, isotope_matrix):
    y_intensity = np.sum(isotope_matrix["y1"]) + np.sum(isotope_matrix["y2"])
    b_intensity = np.sum(isotope_matrix["b1"]) + np.sum(isotope_matrix["b2"])
    p_intensity = np.sum(isotope_matrix["p"])

    return {
        "y_pct": y_intensity/tic,
        "b_pct": b_intensity/tic,
        "p_pct": p_intensity/tic,
        "fragmented_pct": (
            y_intensity + b_intensity
        ) / (y_intensity + b_intensity + p_intensity)
    }

def calculate_peak_features(annotated_spectrum):
    n_peaks = len(annotated_spectrum.spectrum.mz)

    annotated_peaks = 0
    for ion_type in ["b1", "b2", "y1", "y2", "p"]:
        annotated_peaks += np.sum(annotated_spectrum.isotope_matrix[ion_type]>0)
    
    return {
        "n_peaks": n_peaks,
        "pct_peaks_annotated": annotated_peaks/n_peaks
    }
    
def add_features(psm, missfrag_fgen):
    metadata, features = missfrag_fgen._calculate_features(psm=psm, ions="byp")
    features["hyperscore"] = calculate_hyperscore(
        psm=psm,
        mgf_file=missfrag_fgen.mgf_file,
        engine="pyopenms"
    )
    features["tic"] = metadata["annotated_spectrum"].tic
    features["precursor_err_ppm"] = mass_diff(
        psm.peptidoform.theoretical_mz,
        missfrag_fgen.mgf_file.get_by_id(psm["spectrum_id"])["params"]["pepmass"][0],
        mode_is_da=False
    )
    features["precursor_err_da"] = mass_diff(
        psm.peptidoform.theoretical_mz,
        missfrag_fgen.mgf_file.get_by_id(psm["spectrum_id"])["params"]["pepmass"][0],
        mode_is_da=True
    )
    features["ppm_b_mean"], features["ppm_y_mean"] = calculate_ppms(metadata["annotated_spectrum"].ion_mz_delta)
    
    intensity_features = calculate_pct_intensities(
        metadata["annotated_spectrum"].tic, 
        metadata["annotated_spectrum"].isotope_matrix
    )
    features.update(intensity_features)

    features["precursor_peak_present"] = np.sum(metadata["annotated_spectrum"].isotope_matrix["p"]) > 0

    peak_features = calculate_peak_features(metadata["annotated_spectrum"])
    features.update(peak_features)

    psm["rescoring_features"].update(features)

def hyperscore_difference(row, reference):
    try:
        reference_hyperscore = reference[row["spectrum_id"]]
        return float(row["hyperscore"])-float(reference_hyperscore)
    except:
        return None
    
def evaluate_prediction_isobaricity(
        row, ground_truth_peptide, ground_truth_hyperscore
):
    try:
        sequence_match = ground_truth_peptide[
            row["spectrum_id"]
        ] == row["sequence"]
        if sequence_match:
            return "Match"
        
        ref_hyperscore = ground_truth_hyperscore[row["spectrum_id"]]
        if row["hyperscore"] == ref_hyperscore:
            return "Isobaric"
        
        elif row["hyperscore"] > ref_hyperscore:
            return "Better"

        elif row["hyperscore"] < ref_hyperscore:
            return "Worse"

        else:
            return "Error?"

    except:
        return "Unpredicted"

def merge_results(
        root_ground_truth,
        root_denovo_results,
        root_mgf,
        root_out,
        engines,
        ground_truth_filetype="sage",
        only_identified_spectra=True
    ):
    for mgf_path in glob(os.path.join(root_mgf, "*.mgf")):

        # Initalize feature generator for missing fragmentations
        missfrag_fgen = MissingFragmentationSiteFGen(spectrum_path=mgf_path, only_by_ions=False)

        df_list = []

        filename = os.path.basename(mgf_path).split(".")[0]

        # Parse the ground truth result file
        parser = DenovoEngineConverter.select(ground_truth_filetype)
        ground_truth_psmlist = parser.parse(
            os.path.join(root_ground_truth, filename + EXTENSIONS[ground_truth_filetype]),
            mgf_path=mgf_path
        )

        # Filter the ground truth dataset towards only targets (first ranks) that fall below 1% FDR
        if only_identified_spectra:
            ground_truth_psmlist = ground_truth_psmlist[
                (ground_truth_psmlist["qvalue"] < .01) &
                (~ground_truth_psmlist["is_decoy"]) &
                (ground_truth_psmlist["rank"] == 1)
            ]
            spectrum_ids_identified = ground_truth_psmlist["spectrum_id"].tolist()

        print("Adding features to ground truth psmlist...")
        for psm in tqdm(ground_truth_psmlist[:500]):
            add_features(psm, missfrag_fgen)

        ground_truth_df = ground_truth_psmlist.to_dataframe()

        
        df_list.append(ground_truth_df)

        # Parse the denovo results
        for engine in engines:
            parser = DenovoEngineConverter.select(engine)
            psmlist = parser.parse(
                os.path.join(root_denovo_results, engine, f"{filename}.{engine}.csv"),
                mgf_path=mgf_path
            )

            # Add features
            print(f"Adding features to {engine} psmlist...")
            for psm in tqdm(psmlist[:500]):
                add_features(psm, missfrag_fgen)

            df = psmlist.to_dataframe()
    
            # Filter the spectra to only those identified in ground-truth
            if only_identified_spectra:
                df = df[df.spectrum_id.isin(spectrum_ids_identified)]
            
            df_list.append(df)
        
        # General post-processing stuff
        df_results = pd.concat(df_list, ignore_index=True)
        # Equalize the deamidation and I/L as I dont believe there is difference there
        df_results["proforma"] = df_results.peptidoform.apply(lambda x: x.proforma.replace("N[UNIMOD:7]", "D")
                                                                                  .replace("Q[UNIMOD:7]", "E")
                                                                                  .replace("I", "L")
                                                                                  .replace("UNLMOD", "UNIMOD")
                                                            )
    
        # Strip sequence from modification and charge
        df_results["sequence"] = df_results.peptidoform.apply(lambda x: x.sequence.replace("N[UNIMOD:7]", "D")
                                                                                  .replace("Q[UNIMOD:7]", "E")
                                                                                  .replace("I", "L")
                                                                                  .replace("UNLMOD", "UNIMOD")
                                                            )
    
        # Set hyperscore as column
        df_results["hyperscore"] = df_results["rescoring_features"].apply(lambda x: x["hyperscore"])
        df_results["run"] = filename
        df_results["score"] = df_results.apply(collapse_casanovo_score, axis=1)

        
        ground_truth_sequence = ground_truth_df.loc[
            ground_truth_df.source=="sage",
            ["spectrum_id", "sequence"]
        ].set_index("spectrum_id").to_dict()["sequence"]

        ground_truth_hyperscore = ground_truth_df.loc[
            ground_truth_df.source=="sage",
            ["spectrum_id", "hyperscore"]
        ].set_index("spectrum_id").to_dict()["hyperscore"]

        df_results["hyperscore_diff"] = df_results.apply(
            lambda x: hyperscore_difference(x, ground_truth_hyperscore), axis=1
        )
        df_results["match_type"] = df_results.apply(
            lambda x: evaluate_prediction_isobaricity(
                x,
                ground_truth_peptide=ground_truth_sequence,
                ground_truth_hyperscore=ground_truth_hyperscore
            ), axis=1
        )


        df_results = df_results.loc[
            :,
            [
                "spectrum_id",
                "proforma",
                "sequence",
                "match_type",
                "run",
                "source",
                "score",
                "hyperscore",
                "hyperscore_diff",
                "qvalue",
                "is_decoy",
                "metadata",
                "rescoring_features",
                "precursor_mz",
                "retention_time",
                "protein_list"
            ]
        ]
        df_results.to_csv(os.path.join(root_out, filename+".csv"), index=False)

In [5]:
root_ground_truth = "/home/samva/Doctorate/data_directory/PXD028735/search_results/identification"
root_denovo_results = "/home/samva/Doctorate/data_directory/PXD028735/denovo_results"
root_mgf = "/home/samva/Doctorate/data_directory/PXD028735/mgf/Orbitrap_QE/reformatted"
root_out = "/home/samva/Doctorate/data_directory/PXD028735/denovo_results/merged2"
engines = [
    "casanovo",
    "instanovo",
    "contranovo",
    "pepnet",
    "novob"
]

In [6]:
for mgf_path in glob(os.path.join(root_mgf, "*.mgf")):
    ground_truth_filetype = "sage"
    filename = os.path.basename(mgf_path).split(".")[0]
    parser = DenovoEngineConverter.select(ground_truth_filetype)
    ground_truth_psmlist = parser.parse(
        os.path.join(root_ground_truth, filename + EXTENSIONS[ground_truth_filetype]),
        mgf_path=mgf_path
    )
    break

In [7]:
ground_truth_psmlist

PSMList(
    psm_list=[
        PSM(
            peptidoform=Peptidoform('EYIPTVFDNYSAQSAVDGR/2'),
            spectrum_id='controllerType=0 controllerNumber=1 scan=106107',
            run='LFQ_Orbitrap_DDA_Human_01',
            collection=None,
            spectrum=None,
            is_decoy=False,
            score=1.477149,
            qvalue=3.475239e-05,
            pep=None,
            precursor_mz=1066.4993250320701,
            retention_time=104.603325,
            ion_mobility=None,
            protein_list=['sp|P84095|RHOG_HUMAN'],
            rank=1,
            source='sage',
            provenance_data={
                'sage_filename': '/home/samva/Doctorate/data_directory/PXD028735/search_results/identification/LFQ_Orbitrap_DDA_Human_01.sage.tsv'
            },
            metadata={},
            rescoring_features={
                'expmass': 2130.983,
                'calcmass': 2130.9856,
                'peptide_len': 19.0,
                'missed_cleavages': 0.

In [8]:
psmlist = merge_results(
    root_ground_truth=root_ground_truth,
    root_denovo_results=root_denovo_results,
    root_mgf=root_mgf,
    root_out=root_out,
    engines=engines,
    only_identified_spectra=False
)

Adding features to ground truth psmlist...


100%|██████████| 115362/115362 [00:02<00:00, 38619.68it/s]


Adding features to casanovo psmlist...


  6%|▌         | 28/500 [00:00<00:13, 34.13it/s]

/tmp/ipykernel_614447/3537462769.py:38: RuntimeWarning: invalid value encountered in scalar divide
  "fragmented_pct": (
100%|██████████| 114725/114725 [00:02<00:00, 50721.87it/s]


...

Adding features to casanovo psmlist...


 50%|████▉     | 248/500 [00:07<00:07, 32.39it/s]

<Warning: unknown N-terminal modification '-17.027' - adding it to the database> occurred 4 times


 60%|██████    | 300/500 [00:12<00:32,  6.17it/s]/tmp/ipykernel_614447/3537462769.py:38: RuntimeWarning: invalid value encountered in scalar divide
  "fragmented_pct": (
100%|██████████| 500/500 [00:19<00:00, 25.78it/s]


KeyboardInterrupt: 

In [9]:
merged_2 = pd.read_csv("/home/samva/Doctorate/data_directory/PXD028735/denovo_results/merged2/LFQ_Orbitrap_DDA_Human_01.csv")

/tmp/ipykernel_614447/2109589516.py:1: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_2 = pd.read_csv("/home/samva/Doctorate/data_directory/PXD028735/denovo_results/merged2/LFQ_Orbitrap_DDA_Human_01.csv")


In [11]:
merged_2.groupby("source").match_type.value_counts()

source         match_type 
Casanovo4.2.0  Error?         73174
               Unpredicted    24810
               Match          17378
ContraNovo     Error?         35199
               Unpredicted    12307
               Match          10171
InstaNovo      Error?         74751
               Unpredicted    24134
               Match          15840
NovoB          Error?         72486
               Unpredicted    24123
               Match          17458
PepNet         Error?         81319
               Unpredicted    27122
               Match           9804
sage           Match          91189
               Better         53892
               Worse          15394
Name: count, dtype: int64

In [13]:
merged_2[merged_2.spectrum_id=="controllerType=0 controllerNumber=1 scan=100938"]

,spectrum_id,proforma,sequence,match_type,run,source,score,hyperscore,hyperscore_diff,qvalue,is_decoy,metadata,rescoring_features,precursor_mz,retention_time,protein_list
233,controllerType=0 controllerNumber=1 scan=100938,YNVYGYTGQGVFSNVVR/2,YNVYGYTGQGVFSNVVR,Worse,LFQ_Orbitrap_DDA_Human_01,sage,1.405642,39.264406,-11.015353,0.000035,False,{},"{'expmass': 1921.9313, 'calcmass': 1921.932, '...",961.973475,100.49825,['sp|Q13523|PRP4K_HUMAN']
20939,controllerType=0 controllerNumber=1 scan=100938,VVC[UNLMOD:4]VYGGTGLSEQLAELK/2,VVCVYGGTGLSEQLAELK,Match,LFQ_Orbitrap_DDA_Human_01,sage,1.165929,50.279760,0.000000,0.000035,False,{},"{'expmass': 1921.9313, 'calcmass': 1921.982, '...",961.973475,100.49825,['sp|Q7L014|DDX46_HUMAN']
244530,controllerType=0 controllerNumber=1 scan=100938,YNVYGYTGQGVFSNVVR/2,YNVYGYTGQGVFSNVVR,Error?,LFQ_Orbitrap_DDA_Human_01,Casanovo4.2.0,0.987801,NaN,NaN,NaN,NaN,"{'aa_scores': '0.98398,0.98563,0.98571,0.98258...",{},961.972894,6029.89500,NaN
359514,controllerType=0 controllerNumber=1 scan=100938,YNVYGYTGQGVFSNVVR/2,YNVYGYTGQGVFSNVVR,Error?,LFQ_Orbitrap_DDA_Human_01,InstaNovo,-0.084829,NaN,NaN,NaN,NaN,{'scans': '100938'},{},961.972894,6029.89500,NaN
533104,controllerType=0 controllerNumber=1 scan=100938,YNVYGYTGQGVFTNLAR/2,YNVYGYTGQGVFTNLAR,Error?,LFQ_Orbitrap_DDA_Human_01,PepNet,0.279200,NaN,NaN,NaN,NaN,"{'positional_scores': '[0.9991548, 0.9995945, ...",{},961.972894,6029.89500,NaN
649675,controllerType=0 controllerNumber=1 scan=100938,YNVYGYTGQGVFSNVVR/2,YNVYGYTGQGVFSNVVR,Error?,LFQ_Orbitrap_DDA_Human_01,NovoB,0.497128,NaN,NaN,NaN,NaN,"{'ppm_error': '0.003', 'scans': '100938'}",{},961.972894,6029.89500,NaN


In [12]:
merged_2.loc[
    (merged_2.source=="sage") &
    (merged_2.match_type=="Worse")
]

,spectrum_id,proforma,sequence,match_type,run,source,score,hyperscore,hyperscore_diff,qvalue,is_decoy,metadata,rescoring_features,precursor_mz,retention_time,protein_list
233,controllerType=0 controllerNumber=1 scan=100938,YNVYGYTGQGVFSNVVR/2,YNVYGYTGQGVFSNVVR,Worse,LFQ_Orbitrap_DDA_Human_01,sage,1.405642,39.264406,-11.015353,0.000035,False,{},"{'expmass': 1921.9313, 'calcmass': 1921.932, '...",961.973475,100.498250,['sp|Q13523|PRP4K_HUMAN']
723,controllerType=0 controllerNumber=1 scan=68831,LLSNASC[UNLMOD:4]TTNC[UNLMOD:4]LAPLAK/2,LLSNASCTTNCLAPLAK,Worse,LFQ_Orbitrap_DDA_Human_01,sage,1.373265,55.236600,-1.342039,0.000035,False,{},"{'expmass': 1832.916, 'calcmass': 1832.9126, '...",917.465825,75.342384,['sp|P04406|G3P_HUMAN']
820,controllerType=0 controllerNumber=1 scan=90540,VVLMQC[UNLMOD:4]NLESVEEGVK/2,VVLMQCNLESVEEGVK,Worse,LFQ_Orbitrap_DDA_Human_01,sage,1.369510,45.137361,-3.754672,0.000035,False,{},"{'expmass': 1832.9022, 'calcmass': 1832.9011, ...",917.458925,92.317290,['sp|Q9UHY1|NRBP_HUMAN']
1242,controllerType=0 controllerNumber=1 scan=46384,DLPDGPDAPADR/2,DLPDGPDAPADR,Worse,LFQ_Orbitrap_DDA_Human_01,sage,1.355731,45.849336,-2.575259,0.000035,False,{},"{'expmass': 1237.5592, 'calcmass': 1237.5574, ...",619.787425,57.800320,['sp|O95721|SNP29_HUMAN']
1445,controllerType=0 controllerNumber=1 scan=56806,ENPC[UNLMOD:4]QEQGDVLQLK/2,ENPCQEQGDVLQLK,Worse,LFQ_Orbitrap_DDA_Human_01,sage,1.350216,45.300906,-0.977892,0.000035,False,{},"{'expmass': 1656.7799, 'calcmass': 1656.7778, ...",829.397775,65.976030,['sp|Q9H3P2|NELFA_HUMAN']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160092,controllerType=0 controllerNumber=1 scan=112453,SNENPKSEALRSESQKMER/3,SNENPKSEALRSESQKMER,Worse,LFQ_Orbitrap_DDA_Human_01,sage,0.239519,28.848595,-0.064411,0.301701,True,{},"{'expmass': 2220.1694, 'calcmass': 2219.06, 'p...",741.064292,109.750650,['rev_sp|Q6KC79|NIPBL_HUMAN']
160169,controllerType=0 controllerNumber=1 scan=111136,[+43.006]-MVFVEQNLPDGKSNKGR/3,MVFVEQNLPDGKSNKGR,Worse,LFQ_Orbitrap_DDA_Human_01,sage,0.233228,27.944122,-1.077799,0.301960,False,{},"{'expmass': 1960.9143, 'calcmass': 1960.9791, ...",654.645925,108.675410,['tr|A0A484X565|A0A484X565_ECOLX']
160196,controllerType=0 controllerNumber=1 scan=45492,GEPLLASLAAAC[UNLMOD:4]AVSRGTAALLMYR/5,GEPLLASLAAACAVSRGTAALLMYR,Worse,LFQ_Orbitrap_DDA_Human_01,sage,0.231378,29.279609,-1.927278,0.302010,False,{},"{'expmass': 2562.2827, 'calcmass': 2561.3457, ...",513.464365,57.092340,"['tr|A0A2X1KCH6|A0A2X1KCH6_ECOLX', 'tr|A0A2X1Q..."
160339,controllerType=0 controllerNumber=1 scan=28926,EGAPVVDQKVTFSKDFGP/4,EGAPVVDQKVTFSKDFGP,Worse,LFQ_Orbitrap_DDA_Human_01,sage,0.211654,30.187164,-0.498129,0.302368,False,{},"{'expmass': 1920.0236, 'calcmass': 1919.9628, ...",481.013725,43.469470,['tr|A0A377ADZ5|A0A377ADZ5_ECOLX']
